In [6]:
import time

In [26]:
# Chu Thien Hai 20215360
import sys

def change_order_to_come(high, low, old_order):
    tmp = old_order[high]
    for i in range(high, low, -1):
        old_order[i] = old_order[i - 1]
    old_order[low] = tmp

def greedy(file_path, f):
    NMAX = 1005
    e = [0] * NMAX
    l = [0] * NMAX
    d = [0] * NMAX
    c = [[0] * NMAX for _ in range(NMAX)]
    cd = [[0] * NMAX for _ in range(NMAX)]
    route = [0] * NMAX
    time_come = [0] * NMAX
    order_to_come = []
    order_by_l = []
    with open(file_path, 'r') as file:
        data = file.read().split()
        idx = 0
        
        N = int(data[idx])
        idx += 1
        
        for i in range(1, N + 1):
            e[i] = int(data[idx])
            l[i] = int(data[idx + 1])
            d[i] = int(data[idx + 2])
            order_by_l.append((l[i], i))
            idx += 3
        
        for i in range(N + 1):
            for j in range(N + 1):
                c[i][j] = int(data[idx])
                cd[i][j] = c[i][j] + d[j]
                idx += 1
    order_to_come = [0] * NMAX
    order_by_l.sort()
    
    for i in range(1, N + 1):
        order_to_come[i] = order_by_l[i - 1][1]
    
    route[0] = 0
    time_come[0] = 0
    
    for i in range(1, N + 1):
        next_location = order_to_come[i]
        selected_location = next_location
        latest_time_come = l[selected_location]
        selected_time_come = max(e[selected_location], time_come[route[i - 1]] + cd[route[i - 1]][selected_location])
        
        tmp_j = -1
        for j in range(i + 1, N + 1):
            choice = order_to_come[j]
            tmp_time_come = max(e[choice], time_come[route[i - 1]] + cd[route[i - 1]][choice])
            if tmp_time_come + cd[choice][selected_location] < latest_time_come:
                latest_time_come = tmp_time_come + cd[choice][selected_location]
                selected_location = choice
                selected_time_come = tmp_time_come
                tmp_j = j
        
        if selected_location != next_location:
            change_order_to_come(tmp_j, i, order_to_come)
        
        route[i] = selected_location
        time_come[route[i]] = selected_time_come

    f.write(str(N) + '\n')
    for i in range(1, N + 1):
        f.write(str(route[i]) + ' ')



In [27]:
#PYTHON Chu Thien Hai 20215360
# Quy hoạch nguyên tuyến tính

from ortools.linear_solver import pywraplp

def TSP_mixed_integer_programming(n, time_matrix, dist_matrix, f):
    model = pywraplp.Solver.CreateSolver('SCIP')

############## Danh sách input ###############################

    num_nodes = n + 1
    e = {}  # Thời điểm sớm nhất để giao hàng 
    l = {}  # Thời điểm muộn nhất để giao hàng
    d = {}  # Thời gian cần thiết để chuyển hàng
    C = {}  # C[i,j] = d[i] + dist_matrix[i][j]
    d[0] = 0
    e[0] = 0
    l[0] = 1e9
    for i in range(1, num_nodes):
        e[i] = time_matrix[i-1][0]
        l[i] = time_matrix[i-1][1]
        d[i] = time_matrix[i-1][2]
    for i in range(num_nodes):
        for j in range(num_nodes):
            C[i, j] = d[i] + dist_matrix[i][j]

############### Danh sách các biến ##############################

    x = {}  # x[i,j] = 1 if i -> j else 0
    M = {}  # M[i] = thời điểm tới vị trí i
    u = {} # u[i] = j if người i được thăm ở lần thăm thứ j
    w = {} # thời gian chờ đợi tại vị trí i
    t1 = {} 
    t2 = {} 

############### Miền xác định #########################

    for i in range(num_nodes):
        for j in range(num_nodes):
            if j != i:
                x[i, j] = model.IntVar(0, 1, 'x[%i,%i]' % (i, j))
            else:
                x[i, j] = model.IntVar(0,0, 'x[%i,%i]' % (i, j))

    M[0] = model.IntVar(0, 0, 'M[%i]' % 0)
    for i in range(1, num_nodes):
        M[i] = model.IntVar(0, l[i], 'M[%i]' % i)

    u[0] = model.IntVar(1, 1, 'u[%i]' % 0)
    for i in range(1, num_nodes):
        u[i] = model.IntVar(2, num_nodes, 'u[%i]' % i)

    w[0] = model.IntVar(0,0, 'w[%i]' % 0)
    for i in range(1, num_nodes):
        w[i] = model.IntVar(0, l[i], 'w[%i]' % i)


    for i in range(0, num_nodes):
        t1[i] = model.IntVar(0, 1, 't1[%i]' % i)
        t2[i] = model.IntVar(0, 1, 't2[%i]' % i)
    
 ############## Ràng buộc ###############################

    for i in range(num_nodes):
        model.Add(sum(x[i, j] for j in range(num_nodes) if j != i) == 1)
        model.Add(sum(x[j, i] for j in range(num_nodes) if j != i) == 1)

    # subtour elimination
    for i in range(1, num_nodes):
        for j in range(1, num_nodes):
            if i != j:
                model.Add(u[i] - u[j] + 1 <=  num_nodes * (1- x[i, j]))

    # time window
    for i in range(num_nodes): 
        for j in range(1, num_nodes):
            if i != j:
                model.Add(M[j] <= M[i] + w[i] + C[i, j] + (1 - x[i, j]) * 1e9)
                model.Add(M[j] >= M[i] + w[i] + C[i, j] - (1 - x[i, j]) * 1e9)

    # w[i] = max(0, e[i]-M[i])
    for i in range(1, num_nodes):
        model.Add(w[i] >= 0)
        model.Add(w[i] >= e[i] - M[i])
        model.Add(w[i] <= 1e9 * t1[i] + 0)
        model.Add(w[i] <= 1e9 * t2[i] + e[i] - M[i])
        model.Add(t1[i] + t2[i] <= 1)
                
                
############### Objective #################################
    model.Minimize(sum(C[i, j] * x[i, j] for i in range(num_nodes) for j in range(num_nodes) if j != i) + sum(w[i] for i in range(num_nodes)))

############### Solve #####################################
    status = model.Solve()
#     if status == pywraplp.Solver.OPTIMAL:
    f.write(str(n) + '\n')
    route = [0 for _ in range(num_nodes - 1)]
    for i in range(1, num_nodes):
        route[int(u[i].solution_value() - 2)] = i
    for i in range(n):
        f.write(str(route[i]) + ' ')
    
    #     print('Objective value =', model.Objective().Value())
    #     for i in range(num_nodes):
    #         for j in range(num_nodes):
    #             if x[i, j].solution_value() > 0:
    #                 print('x[%i,%i] = %i' % (i, j, x[i, j].solution_value()))
#         for i in range(num_nodes):
#             print('M[%i] = %i' % (i, M[i].solution_value()))
#             print(str(i) + ": " + str(l[i]))
        
    #     for i in range(num_nodes):
    #         print('w[%i] = %i' % (i, w[i].solution_value()))
    #     for i in range(num_nodes):
    #         print('u[%i] = %i' % (i, u[i].solution_value()))

    # return model.Objective().Value()

def ILP(file_path, f):

    input_data = []
    customers = []
    t = []
    with open(file_path, 'r') as file:
    # Read the input
        N = int(file.readline())
        
        for _ in range(1, N+1):
            e, l, d = map(int, file.readline().split())
            customers.append([e, l, d])

        for _ in range(N+1):
            row = list(map(int, file.readline().split()))
            t.append(row)
#     N = int(input())
#     time_matrix = []
#     for i in range(1,N + 1):
#         eld = input().strip().split()
#         e = int(eld[0])
#         l = int(eld[1])
#         d = int(eld[2])
#         time_matrix.append([e,l,d])
#     dist_matrix = [[0 for _ in range(N+1)] for _ in range(N+1)]

#     for i in range(N+1):
#             C = input().strip().split()
#             for j in range(N+1):
#                 dist_matrix[i][j] = int(C[j])

    TSP_mixed_integer_programming(N, customers, t, f)


In [31]:
import os
import time


def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    index = 0
    
    # Number of delivery locations
    N = int(lines[index].strip())
    index += 1
    
    # Time matrix initialization
    time_matrix = [(0, 1e9, 0)]  # Start point time constraints

    # Reading time constraints for each delivery point
    for i in range(1, N + 1):
        e, l, d = map(int, lines[index].strip().split())
        time_matrix.append([e, l, d])
        index += 1

    # Distance matrix initialization
    dist_matrix = []
    for i in range(N + 1):
        dist_matrix.append(list(map(int, lines[index].strip().split())))
        index += 1

    # Number of points in the route
    N_out = int(lines[index].strip())
    index += 1

    # Route output
    route_out = [0] + list(map(int, lines[index].strip().split())) + [0]

    return N, time_matrix, dist_matrix, N_out, route_out

def check_delivery_route(file_path):
    N, time_matrix, dist_matrix, N_out, route_out = read_input_from_file(file_path)

    time_come = 0
    is_valid = True

    for k in range(N_out + 1):
        location = route_out[k]

        if time_come > time_matrix[location][1]:
            print(f"Algorithm does not satisfy constraints at location {location}")
            is_valid = False
            break

        if time_come < time_matrix[location][0]:
            time_come = time_matrix[location][0]

        time_come += time_matrix[location][2]
        time_come += dist_matrix[route_out[k]][route_out[k + 1]]

    if is_valid:
        print(f"Total cost: {time_come}")

if __name__ == "__main__":
    file_p = "/kaggle/input/gentest/TULKH"
    files = os.listdir(file_p)
    for file_name in files:
        if (file_name in ['200.txt', '300.txt', '500.txt', '700.txt', '1000.txt', '100.txt']):
            continue
        file_path = os.path.join(file_p, file_name)
        with open(file_path, 'r') as f:
            content = f.read()
        print(file_name)
        with open('/kaggle/working/tmp', 'w') as f1:
            f1.write(content)
            
        # Replace 'input.txt' with the path to your input file
        with open('/kaggle/working/tmp', 'a') as f1:
            time_begin = time.time()
            CP(file_path, f1)
            time_end = time.time()
            print("Time: ", time_end - time_begin)
        check_delivery_route('/kaggle/working/tmp')
        os.remove('/kaggle/working/tmp')
        print()
    file_p = '/kaggle/input/huststack/hust_stack'
    files = os.listdir(file_p)
    for file_name in files:
        if (file_name in ['input5.txt', 'input10.txt']):
            file_path = os.path.join(file_p, file_name)
            with open(file_path, 'r') as f:
                content = f.read()
            print(file_name)
            with open('/kaggle/working/tmp', 'w') as f1:
                f1.write(content)
            # Replace 'input.txt' with the path to your input file
            with open('/kaggle/working/tmp', 'a') as f1:
                time_begin = time.time()
                CP(file_path, f1)
                time_end = time.time()
                print("Time: ", time_end - time_begin)
            check_delivery_route('/kaggle/working/tmp')
            os.remove('/kaggle/working/tmp')
            print()


50.txt
Time:  0.4942336082458496
Total cost: 200595

80.txt
Time:  1.5493991374969482
Total cost: 500520

70.txt
Time:  0.9395272731781006
Total cost: 442108

60.txt
Time:  0.681359052658081
Total cost: 343812

90.txt
Time:  1.5619440078735352
Total cost: 581126

input5.txt
Time:  0.01047372817993164
Total cost: 465

input10.txt
Time:  0.026499271392822266
Total cost: 779



In [1]:
import random

def read_data_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    n = int(lines[0].strip())
    e, l, d = [], [], []
    for i in range(1, n + 1):
        ei, li, di = map(int, lines[i].strip().split())
        e.append(ei)
        l.append(li)
        d.append(di)

    t = []
    for i in range(n + 1, 2 * n + 2):
        row = list(map(int, lines[i].strip().split()))
        t.append(row)

    return n, e, l, d, t

def print_data_to_file(filename, n, e, l, d, t, best_individual, best_value):
    with open(filename, 'w') as file:
        file.write(f"{n}\n")
        for ei, li, di in zip(e, l, d):
            file.write(f"{ei} {li} {di}\n")
        for row in t:
            file.write(' '.join(map(str, row)) + "\n")
        file.write("Best Individual:\n")
        file.write(" ".join(map(str, best_individual[1:-1])) + "\n")
        file.write(f"Best Value: {best_value}\n")

def evaluate(individual, t, e, l, d, inf):
    n = len(e) - 1
    current_time = 0
    for i in range(n):
        from_node = individual[i]
        to_node = individual[i + 1]
        travel_time = t[from_node][to_node]
        service_time = d[to_node]

        current_time += travel_time
        start_time = e[to_node]
        end_time = l[to_node]

        if current_time > end_time:
            return inf
        if current_time < start_time:
            current_time = start_time

        current_time += service_time

    current_time += t[individual[n]][0]
    return current_time

def generate_init_population(pop_size, n, t, e, l, d, inf, loop_count):
    population = []
    for _ in range(pop_size):
        individual = list(range(1, n + 1))
        random.shuffle(individual)
        i = 0
        while i < loop_count:
            if evaluate([0] + individual + [0], t, e, l, d, inf) < inf:
                break
            random.shuffle(individual)
            i += 1
        population.append([0] + individual + [0])
    return population

def fill_position(child, parent, start, end, n):
    parrent_current = 1
    for current in range(end, n + 1):
        while True:
            if parent[parrent_current] not in child:
                child[current] = parent[parrent_current]
                break
            parrent_current += 1
    for current in range(1, start):
        while True:
            if parent[parrent_current] not in child:
                child[current] = parent[parrent_current]
                break
            parrent_current += 1

def crossover(parent1, parent2, n):
    start, end = sorted(random.sample(range(1, n), 2))

    child1 = [0] + [-1] * n + [0]
    child2 = [0] + [-1] * n + [0]

    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]

    fill_position(child1, parent2, start, end, n)
    fill_position(child2, parent1, start, end, n)

    return child1, child2

def mutate(individual, n, mutation_rate=0.2):
    if random.random() > mutation_rate:
        return individual
    first, second = random.sample(range(1, n), 2)
    individual[first], individual[second] = individual[second], individual[first]
    return individual

def genetic_algorithm(n, e, l, d, t, pop_size=100, max_gen=1000, mutation_rate=0.2, loop_count=10000, inf=9999999):
    e = [0] + e
    l = [0] + l
    d = [0] + d
    population = generate_init_population(pop_size, n, t, e, l, d, inf, loop_count)
    for gen in range(max_gen):
        random.shuffle(population)
        next_population = population
        for i in range(0, len(population), 2):
            parent1, parent2 = population[i], population[i + 1]
            child1, child2 = crossover(parent1, parent2, n)
            mutate(child1, n, mutation_rate)
            mutate(child2, n, mutation_rate)
            next_population.append(child1)
            next_population.append(child2)
        population = sorted(next_population, key=lambda x: evaluate(x, t, e, l, d, inf))[:pop_size]

    best_individual = min(population, key=lambda x: evaluate(x, t, e, l, d, inf))
    return best_individual, evaluate(best_individual, t, e, l, d, inf)

def GA(file_name)
    filename = "dataset/input10.txt"
    output_filename = "output.txt"
    n, e, l, d, t = read_data_from_file(filename)
    best_individual, best_value = genetic_algorithm(n, e, l, d, t)
    print_data_to_file(output_filename, n, e, l, d, t, best_individual, best_value)


10
3 4 6 10 8 7 5 2 9 1


In [7]:
#TSP with Time windows
from ortools.sat.python import cp_model
import time

def TSP_CP(n, time_matrix, dist_matrix, f):
    model = cp_model.CpModel()

    # Define input
    num_nodes = n + 1
    e = {}  # earliest time to visit city i
    l = {}  # latest time to visit city i
    d = {}  # duration time to visit city i
    C = {}  # C[i,j] = d[i] + dist_matrix[i][j]
    
    # Initialize input
    d[0] = 0
    e[0] = 0
    l[0] = int(1e9)
    for i in range(1, num_nodes):
        e[i] = time_matrix[i-1][0]
        l[i] = time_matrix[i-1][1]
        d[i] = time_matrix[i-1][2]
    for i in range(num_nodes):
        for j in range(num_nodes):    
            C[i, j] = d[i] + dist_matrix[i][j]
            
    # Define variables
    x = {}  # x[i,j] = 1 if i -> j else 0
    M = {}  # M[i] = Arrival time at city i
    w = {}  # w[i] = time waiting at city i
    # Initialize variables
    for i in range(num_nodes):
        for j in range(num_nodes):
           x[i, j] = model.NewBoolVar('x[%i,%i]' % (i, j))

    M[0] = model.NewIntVar(0, 0, 'M[%i]' % 0)
    for i in range(1, num_nodes):
        M[i] = model.NewIntVar(0, l[i], 'M[%i]' % i)

    w[0] = model.NewIntVar(0, 0, 'w[%i]' % 0)
    for i in range(1, num_nodes):
        w[i] = model.NewIntVar(0, l[i], 'w[%i]' % i)

    # Define constraints
    # Each city is visited exactly once
    for i in range(num_nodes):
        model.Add(sum(x[i, j] for j in range(num_nodes) if j != i) == 1)
        model.Add(sum(x[j, i] for j in range(num_nodes) if j != i) == 1)
    
    # Time window
    for i in range(num_nodes):
        for j in range(1, num_nodes):
            if i != j:
                model.Add(M[i] + w[i] + C[i, j]*x[i, j] - M[j] <= (1 - x[i, j]) * int(1e9))

    # Waiting time
    for i in range(1, num_nodes):
        model.Add(w[i] >= 0)
        model.Add(w[i] >= e[i] - M[i])
        t1 = model.NewBoolVar('t1[%i]' % i)
        t2 = model.NewBoolVar('t2[%i]' % i)
        model.Add(w[i] <= int(1e9) * t1)
        model.Add(w[i] <= int(1e9) * t2 + e[i] - M[i])
        model.Add(t1 + t2 <= 1)
        
    # Objective
    model.Minimize(sum(C[i, j] * x[i, j] for i in range(num_nodes) for j in range(num_nodes) if j != i) 
                   + sum(w[i] for i in range(num_nodes)))
    
    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        f.write(str(n) + '\n')
        solution = {}
        solution[0] = 0

        for i in range(1, num_nodes):
            solution[i] = solver.Value(M[i])
        solution = sorted(solution.items(), key=lambda x: x[1])

        for i in range(1, num_nodes):
            f.write(str(solution[i][0]) + ' ')
        
                    

def CP(file_path, f):
    time_matrix = []
    dist_matrix = []
    with open(file_path, 'r') as file:
        # Read the input
        N = int(file.readline())

        for i in range(1, N+1):
            e, l, d = map(int, file.readline().split())
            time_matrix.append([e, l, d])

        for i in range(N+1):
            line = list(map(int, file.readline().split()))
            dist_matrix.append(line)
    TSP_CP(N, time_matrix, dist_matrix,f)


In [21]:
#TSP with Time windows
from ortools.sat.python import cp_model
import time

def TSP_CP(n, time_matrix, dist_matrix):
    model = cp_model.CpModel()

    # Define input
    num_nodes = n + 1
    e = {}  # earliest time to visit city i
    l = {}  # latest time to visit city i
    d = {}  # duration time to visit city i
    C = {}  # C[i,j] = d[i] + dist_matrix[i][j]
    
    # Initialize input
    d[0] = 0
    e[0] = 0
    l[0] = 1000000000
    for i in range(1, num_nodes):
        e[i] = time_matrix[i-1][0]
        l[i] = time_matrix[i-1][1]
        d[i] = time_matrix[i-1][2]
    for i in range(num_nodes):
        for j in range(num_nodes):    
            C[i, j] = d[i] + dist_matrix[i][j]
            
    # Define variables
    x = {}  # x[i,j] = 1 if i -> j else 0
    M = {}  # M[i] = time to deliver at city i = Arrival time at city i + time waiting at city i
    w = {}  # time waiting at city i
    
    # Initialize variables
    for i in range(num_nodes):
        for j in range(num_nodes):
            x[i, j] = model.NewIntVar(0, 1, 'x[%i,%i]' % (i, j))

    M[0] = model.NewIntVar(0, 0, 'M[%i]' % 0)
    for i in range(1, num_nodes):
        M[i] = model.NewIntVar(e[i], l[i], 'M[%i]' % i)

    for i in range(num_nodes):
        w[i] = model.NewIntVar(0, l[i] - e[i], 'w[%i]' % i)

    # Define constraints
    # Each city is visited exactly once
    for i in range(num_nodes):
        model.Add(sum(x[i, j] for j in range(num_nodes) if j != i) == 1)
        model.Add(sum(x[j, i] for j in range(num_nodes) if j != i) == 1)
        
    # Time window
    for i in range(num_nodes):
        for j in range(1, num_nodes):
            if i != j:
                model.Add(M[i] + C[i, j]*x[i, j] - M[j] <= (1 - x[i, j]) * 1000000000)
                model.Add(w[j] == M[j] - M[i] - C[i, j]).OnlyEnforceIf(x[i, j])
                
    # Objective
    model.Minimize(sum(C[i, j] * x[i, j] for i in range(num_nodes) for j in range(num_nodes) if j != i) 
                   + sum(w[i] for i in range(num_nodes)))
    
    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

#     if status == cp_model.FEASIBLE:
    print(n)
    solution = {}
    solution[0] = 0

    for i in range(1, num_nodes):
        solution[i] = solver.Value(M[i])
    solution = sorted(solution.items(), key=lambda x: x[1])

    for i in range(1, num_nodes):
        print(solution[i][0], end=' ')
    return solver.ObjectiveValue()
#     else:
#         print(status)
                    

if __name__ == '__main__':

    time_matrix = []
    dist_matrix = []
    with open("/kaggle/input/gentest/TULKH/300.txt", 'r') as file:
        # Read the input
        N = int(file.readline())

        for i in range(1, N+1):
            e, l, d = map(int, file.readline().split())
            time_matrix.append([e, l, d])

        for i in range(N+1):
            line = list(map(int, file.readline().split()))
            dist_matrix.append(line)
    print(TSP_CP(N, time_matrix, dist_matrix))

300


IndexError: list index (90602) out of range

In [25]:
import sys

class HillClimbingSolver:
    def __init__(self, input_filename, output_filename= '/kaggle/working/tmp'):
        self.input_filename = input_filename
        self.output_filename = output_filename
        self.N = 0
        self.e = []
        self.l = []
        self.d = []
        self.c = []
        self.cd = []
        self.route = []

    def read_input(self):
        with open(self.input_filename, 'r') as fin:
            self.N = int(fin.readline().strip())
            self.e = [0] * (self.N + 1)
            self.l = [0] * (self.N + 1)
            self.d = [0] * (self.N + 1)
            self.c = [[0] * (self.N + 1) for _ in range(self.N + 1)]
            self.cd = [[0] * (self.N + 1) for _ in range(self.N + 1)]
            self.route = [0] * (self.N + 1)

            for i in range(1, self.N + 1):
                self.e[i], self.l[i], self.d[i] = map(int, fin.readline().strip().split())
            for i in range(1, self.N + 1):
                line = list(map(int, fin.readline().strip().split()))
                for j in range(1, self.N + 1):
                    self.c[i][j] = line[j-1]
                    self.cd[i][j] = self.c[i][j] + self.d[j]

    def evaluate(self, route):
        currentTime = 0
        totalDistance = 0
        for i in range(1, self.N + 1):
            prev = route[i - 1]
            next = route[i]
            currentTime += self.c[prev][next]
            currentTime = max(currentTime, self.e[next])
            currentTime += self.d[next]
            if currentTime > self.l[next]:
                return sys.maxsize
            totalDistance += self.c[prev][next]
        return totalDistance

    def hill_climbing(self):
        currentRoute = self.route[:self.N + 1]
        currentCost = self.evaluate(currentRoute)
        improved = True

        while improved:
            improved = False
            for i in range(1, self.N):
                for j in range(i + 1, self.N + 1):
                    currentRoute[i], currentRoute[j] = currentRoute[j], currentRoute[i]
                    newCost = self.evaluate(currentRoute)
                    if newCost < currentCost:
                        currentCost = newCost
                        improved = True
                    else:
                        currentRoute[i], currentRoute[j] = currentRoute[j], currentRoute[i]

        with open(self.output_filename, 'a') as fout:
            fout.write(f"{self.N}\n")
            for i in range(1, self.N + 1):
                fout.write(f"{currentRoute[i]} ")
            fout.write("\n")

    def solve(self):
        try:
            self.read_input()
        except FileNotFoundError:
            print("Không thể mở file đầu vào!")
            return 1

        # Khởi tạo lời giải bằng sắp xếp đơn giản
        orderByL = [(self.l[i], i) for i in range(1, self.N + 1)]
        orderByL.sort()
        for i in range(1, self.N + 1):
            self.route[i] = orderByL[i - 1][1]

        try:
            self.hill_climbing()
        except Exception as e:
            print(f"Không thể tạo file đầu ra: {e}")
            return 1

        return 0

def hill_local(file_path):
    solver = HillClimbingSolver(file_path)
    solver.solve()


In [4]:
import numpy as np
from collections import deque
import sys

class TabuSearchVer2:
    def __init__(self, input_filename, output_filename= '/kaggle/working/tmp'):
        self.input_filename = input_filename
        self.output_filename = output_filename
        self.N = 0
        self.e = None
        self.l = None
        self.d = None
        self.c = None
        self.cd = None
        self.route = None
        self.timeCome = None
        self.orderToCome = []
        self.orderByL = []

    def change_order_to_come(self, high, low, old_order):
        tmp = old_order[high]
        for i in range(high, low, -1):
            old_order[i] = old_order[i - 1]
        old_order[low] = tmp

    def evaluate(self, route):
        currentTime = 0
        for i in range(1, self.N + 1):
            customer = route[i]
            currentTime += self.c[route[i - 1]][customer]
            currentTime = max(currentTime, self.e[customer])
            currentTime += self.d[customer]
            if currentTime > self.l[customer]:
                return sys.maxsize  # Vi phạm điều kiện time window
        return currentTime

    def tabu_search(self, route):
        currentCost = self.evaluate(route)
        bestCost = currentCost
        bestRoute = list(route)

        tabuList = deque()
        tabuTenure = 10

        for iteration in range(1000):
            bestNeighbor = list(bestRoute)
            bestNeighborCost = sys.maxsize

            for i in range(1, self.N):
                for j in range(i + 1, self.N + 1):
                    neighbor = list(bestRoute)
                    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

                    if neighbor not in tabuList:
                        neighborCost = self.evaluate(neighbor)
                        if neighborCost < bestNeighborCost:
                            bestNeighbor = neighbor
                            bestNeighborCost = neighborCost

            bestRoute = bestNeighbor
            bestCost = bestNeighborCost

            tabuList.append(list(bestRoute))
            if len(tabuList) > tabuTenure:
                tabuList.popleft()

        for i in range(len(bestRoute)):
            route[i] = bestRoute[i]

    def read_input(self):
        with open(self.input_filename, "r") as fin:
            self.N = int(fin.readline().strip())
            self.e = np.zeros(self.N + 1, dtype=int)
            self.l = np.zeros(self.N + 1, dtype=int)
            self.d = np.zeros(self.N + 1, dtype=int)
            self.c = np.zeros((self.N + 1, self.N + 1), dtype=int)
            self.cd = np.zeros((self.N + 1, self.N + 1), dtype=int)
            for i in range(1, self.N + 1):
                self.e[i], self.l[i], self.d[i] = map(int, fin.readline().strip().split())
                self.orderByL.append((self.l[i], i))
            for i in range(self.N + 1):
                self.c[i] = np.array(list(map(int, fin.readline().strip().split())))
                self.cd[i] = self.c[i] + self.d

    def solve(self):
        self.read_input()
        
        self.route = np.zeros(self.N + 1, dtype=int)
        self.timeCome = np.zeros(self.N + 1, dtype=int)
        self.orderToCome = np.zeros(self.N + 1, dtype=int)
        
        self.orderByL.sort()
        for i in range(1, self.N + 1):
            self.orderToCome[i] = self.orderByL[i - 1][1]

        self.route[0] = 0
        self.timeCome[0] = 0
        for i in range(1, self.N + 1):
            nextLocation = self.orderToCome[i]
            selectedLocation = nextLocation
            lastestTimeCome = self.l[selectedLocation]
            selectedTimeCome = max(self.e[selectedLocation], self.timeCome[self.route[i - 1]] + self.cd[self.route[i - 1]][selectedLocation])
            tmp_j = i
            for j in range(i + 1, self.N + 1):
                choice = self.orderToCome[j]
                tmpTimeCome = max(self.e[choice], self.timeCome[self.route[i - 1]] + self.cd[self.route[i - 1]][choice])
                if tmpTimeCome + self.cd[choice][selectedLocation] < lastestTimeCome:
                    lastestTimeCome = tmpTimeCome + self.cd[choice][selectedLocation]
                    selectedLocation = choice
                    selectedTimeCome = tmpTimeCome
                    tmp_j = j
            if selectedLocation != nextLocation:
                self.change_order_to_come(tmp_j, i, self.orderToCome)
            self.route[i] = selectedLocation
            self.timeCome[self.route[i]] = selectedTimeCome

        # Áp dụng Tabu Search để cải thiện giải pháp từ Greedy
        self.tabu_search(self.route)

        # Ghi kết quả cải tiến từ Tabu Search vào file output
        with open(self.output_filename, "a") as fout:
            fout.write(f"{self.N}\n")
            for i in range(1, self.N + 1):
                fout.write(f"{self.route[i]} ")
            fout.write("\n")

def tabu_v2(file_path):
    solver = TabuSearchVer2(file_path)
    solver.solve()


In [5]:
import sys
import numpy as np
from collections import deque

class TabuSearchVer1:
    def __init__(self, input_filename, output_filename = '/kaggle/working/tmp'):
        self.input_filename = input_filename
        self.output_filename = output_filename
        self.N = 0
        self.e = None
        self.l = None
        self.d = None
        self.c = None
        self.cd = None
        self.route = None

    def read_input(self):
        with open(self.input_filename, 'r') as fin:
            self.N = int(fin.readline().strip())
            self.e = np.zeros(self.N + 1, dtype=int)
            self.l = np.zeros(self.N + 1, dtype=int)
            self.d = np.zeros(self.N + 1, dtype=int)
            self.c = np.zeros((self.N + 1, self.N + 1), dtype=int)
            self.cd = np.zeros((self.N + 1, self.N + 1), dtype=int)
            for i in range(1, self.N + 1):
                self.e[i], self.l[i], self.d[i] = map(int, fin.readline().strip().split())
            for i in range(self.N + 1):
                self.c[i] = np.array(list(map(int, fin.readline().strip().split())))
                self.cd[i] = self.c[i] + self.d

    def evaluate(self, route):
        currentTime = 0
        totalDistance = 0
        for i in range(1, self.N + 1):
            prev = route[i - 1]
            next = route[i]
            currentTime += self.c[prev][next]
            currentTime = max(currentTime, self.e[next])
            currentTime += self.d[next]
            if currentTime > self.l[next]:
                return sys.maxsize
            totalDistance += self.c[prev][next]
        return totalDistance

    def tabu_search(self, fout):
        currentRoute = np.copy(self.route[:self.N + 1])
        currentCost = self.evaluate(currentRoute)
        bestRoute = np.copy(currentRoute)
        bestCost = currentCost

        tabuList = deque()
        tabuTenure = 10
        maxIterations = 1000
        iteration = 0

        while iteration < maxIterations:
            bestNeighbor = np.copy(currentRoute)
            bestNeighborCost = sys.maxsize

            for i in range(1, self.N + 1):
                for j in range(i + 1, self.N + 1):
                    neighbor = np.copy(currentRoute)
                    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

                    if not any(np.array_equal(neighbor, t) for t in tabuList):
                        neighborCost = self.evaluate(neighbor)
                        if neighborCost < bestNeighborCost:
                            bestNeighbor = np.copy(neighbor)
                            bestNeighborCost = neighborCost

            if bestNeighborCost < bestCost:
                bestRoute = np.copy(bestNeighbor)
                bestCost = bestNeighborCost

            currentRoute = np.copy(bestNeighbor)
            currentCost = bestNeighborCost

            tabuList.append(np.copy(currentRoute))
            if len(tabuList) > tabuTenure:
                tabuList.popleft()

            iteration += 1

        fout.write(f"{self.N}\n")
        for i in range(1, self.N + 1):
            fout.write(f"{bestRoute[i]} ")
        fout.write("\n")

    def solve(self):
        try:
            self.read_input()
        except FileNotFoundError:
            print("Không thể mở file đầu vào!", file=sys.stderr)
            return 1

        self.route = np.zeros(self.N + 1, dtype=int)
        orderByL = [(self.l[i], i) for i in range(1, self.N + 1)]
        orderByL.sort()
        for i in range(1, self.N + 1):
            self.route[i] = orderByL[i - 1][1]

        try:
            with open(self.output_filename, 'a') as fout:
                self.tabu_search(fout)
        except IOError:
            print("Không thể tạo file đầu ra!", file=sys.stderr)
            return 1

        return 0

def tabuv1(file_path):
    solver = TabuSearchVer1(file_path)
    solver.solve()


In [6]:
import sys
import random
import math

class SimulatedAnnealingSolver:
    def __init__(self, input_filename, output_filename = '/kaggle/working/tmp', initial_temperature=1000, cooling_rate=0.995):
        self.input_filename = input_filename
        self.output_filename = output_filename
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.N = 0
        self.e = []
        self.l = []
        self.d = []
        self.c = []
        self.cd = []
        self.route = []

    def read_input(self):
        with open(self.input_filename, 'r') as fin:
            self.N = int(fin.readline().strip())
            self.e = [0] * (self.N + 1)
            self.l = [0] * (self.N + 1)
            self.d = [0] * (self.N + 1)
            self.c = [[0] * (self.N + 1) for _ in range(self.N + 1)]
            self.cd = [[0] * (self.N + 1) for _ in range(self.N + 1)]
            self.route = [0] * (self.N + 1)

            for i in range(1, self.N + 1):
                self.e[i], self.l[i], self.d[i] = map(int, fin.readline().strip().split())
            for i in range(1, self.N + 1):
                line = list(map(int, fin.readline().strip().split()))
                for j in range(1, self.N + 1):
                    self.c[i][j] = line[j-1]
                    self.cd[i][j] = self.c[i][j] + self.d[j]

    def evaluate(self, route):
        currentTime = 0
        totalDistance = 0
        for i in range(1, self.N + 1):
            prev = route[i - 1]
            next = route[i]
            currentTime += self.c[prev][next]
            currentTime = max(currentTime, self.e[next])
            currentTime += self.d[next]
            if currentTime > self.l[next]:
                return sys.maxsize
            totalDistance += self.c[prev][next]
        return totalDistance

    def simulated_annealing(self):
        currentRoute = self.route[:self.N + 1]
        currentCost = self.evaluate(currentRoute)
        bestRoute = currentRoute[:]
        bestCost = currentCost
        temperature = self.initial_temperature

        while temperature > 1:
            i, j = random.sample(range(1, self.N + 1), 2)
            currentRoute[i], currentRoute[j] = currentRoute[j], currentRoute[i]
            newCost = self.evaluate(currentRoute)

            if newCost < currentCost or math.exp((currentCost - newCost) / temperature) > random.random():
                currentCost = newCost
                if currentCost < bestCost:
                    bestCost = currentCost
                    bestRoute = currentRoute[:]
            else:
                currentRoute[i], currentRoute[j] = currentRoute[j], currentRoute[i]

            temperature *= self.cooling_rate

        self.route = bestRoute
        return bestCost

    def solve(self):
        try:
            self.read_input()
        except FileNotFoundError:
            print("Không thể mở file đầu vào!")
            return 1

        # Khởi tạo lời giải bằng sắp xếp đơn giản
        orderByL = [(self.l[i], i) for i in range(1, self.N + 1)]
        orderByL.sort()
        for i in range(1, self.N + 1):
            self.route[i] = orderByL[i - 1][1]

        try:
            self.simulated_annealing()
            self.write_output()
        except Exception as e:
            print(f"Không thể tạo file đầu ra: {e}")
            return 1

        return 0

    def write_output(self):
        with open(self.output_filename, 'a') as fout:
            fout.write(f"{self.N}\n")
            for i in range(1, self.N + 1):
                fout.write(f"{self.route[i]} ")
            fout.write("\n")

def aneal(file_path):
    solver = SimulatedAnnealingSolver(file_path)
    solver.solve()


In [5]:
class Solver:
    def __init__(self, filename):
        self.MAX_N = 1000
        self.N = 0
        self.e = [0] * (self.MAX_N + 1)
        self.l = [0] * (self.MAX_N + 1)
        self.d = [0] * (self.MAX_N + 1)
        self.t = [[0] * (self.MAX_N + 1) for _ in range(self.MAX_N + 1)]
        self.s = [0] * (self.MAX_N + 1)
        self.visited = [False] * (self.MAX_N + 1)
        self.C = [[0] * (self.MAX_N + 1) for _ in range(self.MAX_N + 1)]
        self.M = [0] * (self.MAX_N + 1)
        self.BEST = 1000000000
        self.path = [0] * (self.MAX_N + 1)
        self.Cmin = 1000000000

        self.input(filename)

    def input(self, filename):
        with open(filename, 'r') as fin:
            self.N = int(fin.readline().strip())
            for i in range(1, self.N + 1):
                self.e[i], self.l[i], self.d[i] = map(int, fin.readline().strip().split())
            for i in range(self.N + 1):
                self.t[i] = list(map(int, fin.readline().strip().split()))

        # Initialize C matrix and find Cmin
        for i in range(self.N + 1):
            for j in range(self.N + 1):
                self.C[i][j] = self.d[i] + self.t[i][j]
                if i != j:
                    self.Cmin = min(self.Cmin, self.C[i][j])

    def solution(self):
        temp_total = self.M[self.s[self.N]] + self.C[self.s[self.N]][0]
        if temp_total < self.BEST:
            self.BEST = temp_total
            for i in range(self.N + 1):
                self.path[i] = self.s[i]

    def TRY(self, k, prev):
        for i in range(1, self.N + 1):
            if not self.visited[i] and self.M[prev] + self.C[prev][i] <= self.l[i]:
                self.visited[i] = True
                self.s[k] = i
                M_prev = self.M[i]
                if self.M[prev] + self.C[prev][i] < self.e[i]:
                    self.M[i] = self.e[i]
                else:
                    self.M[i] = self.M[prev] + self.C[prev][i]
                if self.M[i] > self.BEST - self.Cmin * (self.N - k + 1):
                    self.visited[i] = False
                    self.M[i] = M_prev
                    continue
                if k == self.N:
                    self.solution()
                else:
                    self.TRY(k + 1, i)
                self.visited[i] = False
                self.M[i] = M_prev
        return 1000000000

    def solve(self):
        self.s[0] = 0
        self.M[0] = 0
        self.TRY(1, 0)

    def output(self, output_filename = '/kaggle/working/tmp' ):
        with open(output_filename, 'a') as fout:
            fout.write(f"{self.N}\n")
            fout.write(" ".join(map(str, self.path[1:self.N + 1])))


def complete_s(file_path):
    solver = Solver(file_path)
    solver.solve()
    solver.output()




In [21]:
#TSP with Time windows
from ortools.sat.python import cp_model
import time

def TSP_CP(n, time_matrix, dist_matrix, f):
    model = cp_model.CpModel()

    # Define input
    num_nodes = n + 1
    e = {}  # earliest time to visit city i
    l = {}  # latest time to visit city i
    d = {}  # duration time to visit city i
    C = {}  # C[i,j] = d[i] + dist_matrix[i][j]
    
    # Initialize input
    d[0] = 0
    e[0] = 0
    l[0] = int(1e9)
    for i in range(1, num_nodes):
        e[i] = time_matrix[i-1][0]
        l[i] = time_matrix[i-1][1]
        d[i] = time_matrix[i-1][2]
    for i in range(num_nodes):
        for j in range(num_nodes):    
            C[i, j] = d[i] + dist_matrix[i][j]
            
    # Define variables
    x = {}  # x[i,j] = 1 if i -> j else 0
    M = {}  # M[i] = Arrival time at city i
    w = {}  # w[i] = time waiting at city i
    # Initialize variables
    for i in range(num_nodes):
        for j in range(num_nodes):
           x[i, j] = model.NewBoolVar('x[%i,%i]' % (i, j))

    M[0] = model.NewIntVar(0, 0, 'M[%i]' % 0)
    for i in range(1, num_nodes):
        M[i] = model.NewIntVar(0, l[i], 'M[%i]' % i)

    w[0] = model.NewIntVar(0, 0, 'w[%i]' % 0)
    for i in range(1, num_nodes):
        w[i] = model.NewIntVar(0, l[i], 'w[%i]' % i)

    # Define constraints
    # Each city is visited exactly once
    for i in range(num_nodes):
        model.Add(sum(x[i, j] for j in range(num_nodes) if j != i) == 1)
        model.Add(sum(x[j, i] for j in range(num_nodes) if j != i) == 1)
    
    # Time window
    for i in range(num_nodes):
        for j in range(1, num_nodes):
            if i != j:
                model.Add(M[i] + w[i] + C[i, j]*x[i, j] - M[j] <= (1 - x[i, j]) * int(1e9))

    # Waiting time
    for i in range(1, num_nodes):
        model.Add(w[i] >= 0)
        model.Add(w[i] >= e[i] - M[i])
        t1 = model.NewBoolVar('t1[%i]' % i)
        t2 = model.NewBoolVar('t2[%i]' % i)
        model.Add(w[i] <= int(1e9) * t1)
        model.Add(w[i] <= int(1e9) * t2 + e[i] - M[i])
        model.Add(t1 + t2 <= 1)
        
    # Objective
    model.Minimize(sum(C[i, j] * x[i, j] for i in range(num_nodes) for j in range(num_nodes) if j != i) 
                   + sum(w[i] for i in range(num_nodes)))
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        f.write(str(n) + '\n')
        solution = {}
        solution[0] = 0
        for i in range(1, num_nodes):
            solution[i] = solver.Value(M[i])
        solution = sorted(solution.items(), key=lambda x: x[1])
        for i in range(1, n+1):
            f.write(str(solution[i][0]) + ' ')


                    

def CP(file_path, f):

    # input from file

    time_matrix = []
    dist_matrix = []
    with open(file_path, 'r') as file:
        # Read the input
        N = int(file.readline())

        for i in range(1, N+1):
            e, l, d = map(int, file.readline().split())
            time_matrix.append([e, l, d])

        for i in range(N+1):
            line = list(map(int, file.readline().split()))
            dist_matrix.append(line)
    TSP_CP(N, time_matrix, dist_matrix,f)